# Enhancing E-Commerce Strategies Through Customer Behavior Analysis
<span style="font-size:20px;"> - Amisha Kelkar, Arundhati (Ari) Kolahal, Chaitali Deshmukh, Neha Shastri

## Data Cleaning and Preparation

### Importing Relevant Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Loading the Dataset

In [ ]:
events = pd.read_csv('events.csv')
events.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   timestamp      2756101 non-null  int64  
 1   visitorid      2756101 non-null  int64  
 2   event          2756101 non-null  object 
 3   itemid         2756101 non-null  int64  
 4   transactionid  22457 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


### Preprocessing Data and Feature Engineering

* The `timestamp` column in the events is converted to datetime from timestamp.
* The number of events have been aggregated per user
* The number of unique items purchased by every visitor is counted.

In [ ]:
# Convert timestamp to datetime
events['timestamp'] = pd.to_datetime(events['timestamp'], unit='ms')

# Aggregate counts per visitor
event_counts = events.pivot_table(index='visitorid', columns='event', values='itemid', aggfunc='count', fill_value=0)
event_counts.columns = ['total_addtocart', 'total_transaction', 'total_view']

# Count unique items per visitor
unique_items = events.groupby('visitorid')['itemid'].nunique().reset_index(name='unique_items')

* The `compute_avg_time_diffs` computes time differnece in seconds between each of the event i.e. view, add_to_cart and transactions
* The `compute_avg_transaction_gap`  computes average time between transactions
* The `process_visitor_batch` processes the data in batches to run the above functions in order to compute the features calculaed in previous functions
* Since the `events` data is large, using the defined functions, the data is processed in batches of 10,000 visitors and results are concatenated to a final dataframe `final_df` which is saved in parquet format for future use. The parquet file was used subsequently due to computational complexity.

In [ ]:
#DO NOT RUN

#I REPEAT. DO NOT RUN!!!!
from joblib import Parallel, delayed

# Function to compute time differences for event pairs
def compute_avg_time_diffs(ordered_events, event1, event2):
    timestamps_1 = ordered_events.loc[ordered_events['event'] == event1, 'timestamp'].values
    timestamps_2 = ordered_events.loc[ordered_events['event'] == event2, 'timestamp'].values

    # Match each event1 to the next occurring event2
    time_diffs = []
    i, j = 0, 0
    while i < len(timestamps_1) and j < len(timestamps_2):
        if timestamps_2[j] > timestamps_1[i]:  # Ensure sequential ordering
            time_diffs.append(timestamps_2[j] - timestamps_1[i])
            i += 1
        j += 1  # Move to the next add_to_cart or transaction event

    time_diffs = np.array(time_diffs) / 1e9  # Convert nanoseconds to seconds
    return np.mean(time_diffs) if len(time_diffs) > 0 else np.nan

# Function to compute time between transactions
def compute_avg_transaction_gap(ordered_transactions):
    if len(ordered_transactions) < 2:
        return np.nan

    time_diffs = np.diff(ordered_transactions) / 1e9  # Convert to seconds
    return np.mean(time_diffs) if len(time_diffs) > 0 else np.nan

# Process a batch of visitors
def process_visitor_batch(batch_id, visitor_ids):
    batch_results = []

    for visitor_id in visitor_ids:
        visitor_data = df[df['visitorid'] == visitor_id].sort_values(by='timestamp')  # Ensure correct event order

        avg_view_to_addcart = compute_avg_time_diffs(visitor_data, 'view', 'addtocart')
        avg_addcart_to_transaction = compute_avg_time_diffs(visitor_data, 'addtocart', 'transaction')

        transaction_times = visitor_data.loc[visitor_data['event'] == 'transaction', 'timestamp'].values
        avg_transaction_gap = compute_avg_transaction_gap(transaction_times)

        batch_results.append({
            'visitorid': visitor_id,
            'avg_time_view_to_addtocart': avg_view_to_addcart,
            'avg_time_addtocart_to_transaction': avg_addcart_to_transaction,
            'avg_time_between_transactions': avg_transaction_gap
        })

    # Save batch results as Parquet
    batch_filename = f'batch_results_{batch_id}.parquet'
    pd.DataFrame(batch_results).to_parquet(batch_filename)
    return batch_filename

# Load DataFrame and ensure timestamps are sorted
df = events.sort_values(by=['visitorid', 'timestamp'])  # Sort entire dataset once

# Split visitors into chunks
visitor_ids = df['visitorid'].unique()
chunk_size = 10_000
visitor_chunks = [visitor_ids[i:i + chunk_size] for i in range(0, len(visitor_ids), chunk_size)]

# Parallel processing
batch_files = Parallel(n_jobs=-1)(
    delayed(process_visitor_batch)(i, chunk) for i, chunk in enumerate(visitor_chunks)
)

# Combine all results
final_df = pd.concat([pd.read_parquet(file) for file in batch_files])

# Save final results
final_df.to_parquet("final_visitor_metrics.parquet")

print("Processing complete. Results saved in 'final_visitor_metrics.parquet'.")

In [ ]:
final_df = pd.read_parquet("final_visitor_metrics.parquet")

* The week is extracted from the timestamp in the `events` dataset.
* The average interactions per user in a week been calculated.
* The average items per transaction for every user in a week has been calculated.

In [ ]:
# Compute frequency of interactions in a week
events['week'] = events['timestamp'].dt.to_period('W')

interaction_stats = (
    events.groupby('visitorid')
    .agg(total_interactions=('week', 'count'), unique_weeks=('week', 'nunique'))
    .assign(avg_interactions_per_week=lambda x: x['total_interactions'] / x['unique_weeks'])
    .reset_index()
)
w
# Average number of items per transaction
avg_items_per_transaction = events[events['event'] == 'transaction'].groupby('visitorid')['transactionid'].count().reset_index()
avg_items_per_transaction = avg_items_per_transaction.groupby('visitorid')['transactionid'].mean().reset_index(name='avg_items_per_transaction')

In [ ]:
interaction_stats.head(3)

,visitorid,total_interactions,unique_weeks,avg_interactions_per_week
0,0,3,1,3.0
1,1,1,1,1.0
2,2,8,1,8.0


* In order to get more insight in the customer behaviour patterns, conditional probability of events have been calculated.
* The conditional probability of a user adding an item to the cart when an item was viewed has been calculated.
* The conditional probability of a user actually getting through with the transaction when an item has been added to cart has been calculated.

In [ ]:
conditional_probs = events.groupby(['visitorid', 'itemid', 'event']).size().unstack(fill_value=0)

# Compute conditional probabilities
conditional_probs['p_addtocart_given_view'] = conditional_probs['addtocart'] / conditional_probs['view'].replace(0, 1)
conditional_probs['p_transaction_given_addtocart'] = conditional_probs['transaction'] / conditional_probs['addtocart'].replace(0, 1)
conditional_probs = conditional_probs.groupby('visitorid')[['p_addtocart_given_view', 'p_transaction_given_addtocart']].mean()

Probabilty should not be greater than 1. Therefore we are capping it at 1.

In [ ]:
# Merge all computed features
users = event_counts.merge(unique_items, on='visitorid', how='left')\
                        .merge(final_df, on='visitorid', how='left')\
                        .merge(interaction_stats[['avg_interactions_per_week', 'visitorid']], on='visitorid', how='left')\
                        .merge(conditional_probs, on='visitorid', how='left')\
                        .merge(avg_items_per_transaction, on='visitorid', how='left')

# Display the final DataFrame
print(users.head())

   visitorid  total_addtocart  total_transaction  total_view  unique_items  \
0          0                0                  0           3             3   
1          1                0                  0           1             1   
2          2                0                  0           8             4   
3          3                0                  0           1             1   
4          4                0                  0           1             1   

   avg_time_view_to_addtocart  avg_time_addtocart_to_transaction  \
0                         NaN                                NaN   
1                         NaN                                NaN   
2                         NaN                                NaN   
3                         NaN                                NaN   
4                         NaN                                NaN   

   avg_time_between_transactions  avg_interactions_per_week  \
0                            NaN                        3.0

In [ ]:
users.head(3)

,visitorid,total_addtocart,total_transaction,total_view,unique_items,avg_time_view_to_addtocart,avg_time_addtocart_to_transaction,avg_time_between_transactions,avg_interactions_per_week,p_addtocart_given_view,p_transaction_given_addtocart,avg_items_per_transaction
0,0,0,0,3,3,NaN,NaN,NaN,3.0,0.0,0.0,NaN
1,1,0,0,1,1,NaN,NaN,NaN,1.0,0.0,0.0,NaN
2,2,0,0,8,4,NaN,NaN,NaN,8.0,0.0,0.0,NaN


In [ ]:
users = users.drop(['avg_time_addtocart_to_transaction', 'avg_time_view_to_addtocart', 'avg_time_between_transactions', 'avg_items_per_transaction'], axis = 1)

In [ ]:
missing_values = users.isnull().sum()
print("Missing Values Summary:")
print(missing_values[missing_values > 0])

Missing Values Summary:
Series([], dtype: int64)


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407580 entries, 0 to 1407579
Data columns (total 8 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   visitorid                      1407580 non-null  int64  
 1   total_addtocart                1407580 non-null  int64  
 2   total_transaction              1407580 non-null  int64  
 3   total_view                     1407580 non-null  int64  
 4   unique_items                   1407580 non-null  int64  
 5   avg_interactions_per_week      1407580 non-null  float64
 6   p_addtocart_given_view         1407580 non-null  float64
 7   p_transaction_given_addtocart  1407580 non-null  float64
dtypes: float64(3), int64(5)
memory usage: 85.9 MB


In [ ]:
users.to_parquet('users.parquet')

In [ ]:
# Filter rows where p_addtocart_given_view > 1
invalid_entries = conditional_probs[conditional_probs['p_addtocart_given_view'] > 1].reset_index()

# Display the first 3 rows
invalid_entries.head(3)

event,visitorid,p_addtocart_given_view,p_transaction_given_addtocart
0,1399,2.000000,0.000000
1,1634,1.500000,0.000000
2,6958,2.317553,0.003502


In [ ]:
invalid_entries = conditional_probs[conditional_probs['p_transaction_given_addtocart'] > 1].reset_index()

# Display the first 3 rows
invalid_entries.head(3)

event,visitorid,p_addtocart_given_view,p_transaction_given_addtocart
0,30374,0.333333,2.0
1,47029,1.000000,2.0
2,73567,0.333333,2.0


In [ ]:
# Cap probabilities at 1
conditional_probs['p_addtocart_given_view'] = conditional_probs['p_addtocart_given_view'].clip(upper=1)
conditional_probs['p_transaction_given_addtocart'] = conditional_probs['p_transaction_given_addtocart'].clip(upper=1)

In [ ]:
users.to_parquet('users_w_capped_probabilities.parquet')

The original `events` table and the newly created `users` table are further used in the downstream tasks.